In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
from LagsCreator import LagsCreator

In [ ]:
data = np.array([["a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8", "a9", "a10", "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20"], 
                 ["b1", "b2", "b3", "b4", "b5", "b6", "b7", "b8", "b9", "b10", "b11", "b12", "b13", "b14", "b15", "b16", "b17", "b18", "b19", "b20"], 
                 ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11", "c12", "c13", "c14", "c15", "c16", "c17", "c18", "c19", "c20"]])

In [ ]:
df = pd.DataFrame(np.transpose(data), index = pd.date_range("2018-08-22", periods = 20), columns = pd.MultiIndex.from_product([["Group 1"], ["A", "B", "C"]], names = ["Group", "Time-series"]))

In [ ]:
df

In [ ]:
lags_dict = dict()
# Define lags for each indicator.
lags_dict["A"] = 3
lags_dict["B"] = 4
lags_dict["C"] = 0

In [ ]:
creator = LagsCreator(df, lags_dict, "A")

In [ ]:
X_train, y_train, X_val, y_val, X_test = creator.to_supervised(4, single_step=False,return_dataframe = True,h=1, 
                                                               validation=True, feature_time=False, return_single_level = True)

In [ ]:
X_train

In [ ]:
y_train

In [11]:
X_test

Features      A                  B                       C
Lags     x(t-2) x(t-1) x(t) x(t-3) x(t-2) x(t-1) x(t)    x
0           a18    a19  a20    b17    b18    b19  b20  c20

In [12]:
train, val, test = creator.visualization(boundaries = True)

In [13]:
len(train)

4

In [14]:
train[2]

Time-series,A,B,C
2018-08-22 00:00:00,a1,b1,c1
2018-08-23 00:00:00,a2,b2,c2
2018-08-24 00:00:00,a3,b3,c3
2018-08-25 00:00:00,a4,b4,c4
2018-08-26 00:00:00,a5,b5,c5
2018-08-27 00:00:00,a6,b6,c6
2018-08-28 00:00:00,a7,b7,c7
2018-08-29 00:00:00,a8,b8,c8
2018-08-30 00:00:00,a9,b9,c9
2018-08-31 00:00:00,a10,b10,c10


In [15]:
val[-1]

Time-series,A,B,C
2018-08-22 00:00:00,a1,b1,c1
2018-08-23 00:00:00,a2,b2,c2
2018-08-24 00:00:00,a3,b3,c3
2018-08-25 00:00:00,a4,b4,c4
2018-08-26 00:00:00,a5,b5,c5
2018-08-27 00:00:00,a6,b6,c6
2018-08-28 00:00:00,a7,b7,c7
2018-08-29 00:00:00,a8,b8,c8
2018-08-30 00:00:00,a9,b9,c9
2018-08-31 00:00:00,a10,b10,c10


In [16]:
test[0]

Time-series,A,B,C
2018-08-22 00:00:00,a1,b1,c1
2018-08-23 00:00:00,a2,b2,c2
2018-08-24 00:00:00,a3,b3,c3
2018-08-25 00:00:00,a4,b4,c4
2018-08-26 00:00:00,a5,b5,c5
2018-08-27 00:00:00,a6,b6,c6
2018-08-28 00:00:00,a7,b7,c7
2018-08-29 00:00:00,a8,b8,c8
2018-08-30 00:00:00,a9,b9,c9
2018-08-31 00:00:00,a10,b10,c10
